In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

# making all pandas columns visable with display command
pd.set_option('display.max_columns', 500)

In [2]:
# df_EPL = pd.read_csv('datasets/EPL.csv', index_col = 0)
df_EPL = pd.read_csv('datasets/EPL_sort.csv', index_col = 0)
df_EPL

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2,1731,2008/2009,1,2008-08-16 00:00:00,489044,8472,8650,0,1,5.50,3.60,1.67,0,0,2,0,13,12,4,11,3,5,15,19,1,8,46,54,0,0,H
6,1735,2008/2009,1,2008-08-16 00:00:00,489048,8549,8586,2,1,3.20,3.40,2.25,1,0,2,0,11,11,7,2,5,5,25,27,7,9,53,47,0,0,H
7,1736,2008/2009,1,2008-08-16 00:00:00,489049,8559,10194,3,1,1.83,3.50,4.50,1,0,2,0,14,15,5,1,7,6,22,29,4,3,47,53,0,0,A
3,1732,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.40,4.20,2,0,1,0,14,13,5,7,7,15,15,27,6,10,52,48,0,0,H
8,1737,2008/2009,1,2008-08-16 00:00:00,489050,8667,9879,2,1,2.60,3.20,2.80,3,0,0,0,10,9,5,4,5,6,16,14,5,6,53,47,0,0,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,4699,2015/2016,38,2016-05-15 00:00:00,1987597,9825,10252,4,0,1.17,9.00,17.00,0,0,1,0,10,6,7,4,8,0,16,6,5,4,60,40,48,62,H
2977,4706,2015/2016,38,2016-05-15 00:00:00,1987604,10003,8456,1,1,6.00,4.75,1.53,1,0,3,0,5,19,1,5,2,12,14,15,4,4,50,50,46,65,D
2976,4705,2015/2016,38,2016-05-15 00:00:00,1987603,10194,8654,2,1,3.50,3.60,2.15,3,0,1,0,8,12,3,10,2,7,13,18,4,10,45,55,44,38,D
2974,4703,2015/2016,38,2016-05-15 00:00:00,1987601,10261,8586,5,1,4.50,4.00,1.80,1,1,2,0,7,6,11,8,4,8,12,14,3,8,39,61,42,59,D


In [3]:
df_EPL.sort_values('date',ascending=False)

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2973,4702,2015/2016,38,2016-05-17 00:00:00,1987600,10260,8678,3,1,1.67,4.20,5.25,1,0,0,0,9,8,5,5,4,2,16,5,7,1,68,32,63,42,H
2978,4707,2015/2016,38,2016-05-15 00:00:00,1987605,9817,8472,2,2,2.05,3.75,3.70,1,0,0,0,10,9,6,2,10,2,9,12,7,4,49,51,60,42,H
2975,4704,2015/2016,38,2016-05-15 00:00:00,1987602,8466,9826,4,1,1.36,5.50,9.00,1,0,2,0,6,14,4,5,13,3,20,15,2,5,49,51,49,80,D
2979,4708,2015/2016,38,2016-05-15 00:00:00,1987606,8659,8650,1,1,2.40,3.60,3.00,3,0,1,0,12,13,5,3,8,3,12,14,7,5,33,67,44,34,H
2972,4701,2015/2016,38,2016-05-15 00:00:00,1987599,8668,9850,3,0,1.80,4.00,4.50,0,0,0,0,6,10,9,4,4,5,17,17,4,11,47,53,34,70,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1732,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.40,4.20,2,0,1,0,14,13,5,7,7,15,15,27,6,10,52,48,0,0,H
6,1735,2008/2009,1,2008-08-16 00:00:00,489048,8549,8586,2,1,3.20,3.40,2.25,1,0,2,0,11,11,7,2,5,5,25,27,7,9,53,47,0,0,H
5,1734,2008/2009,1,2008-08-16 00:00:00,489047,8668,8655,2,3,2.00,3.30,4.00,2,0,2,0,11,11,2,8,7,8,14,21,3,4,51,49,0,0,A
1,1730,2008/2009,1,2008-08-16 00:00:00,489043,9825,8659,1,0,1.20,6.50,15.00,0,0,0,0,11,9,12,2,13,3,21,7,7,5,66,34,0,0,H


In [4]:
current_match_id = 1987598
current_match = df_EPL[df_EPL['match_api_id'] == current_match_id]
current_match

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2971,4700,2015/2016,38,2016-05-15 00:00:00,1987598,8455,8197,1,1,2.3,3.75,3.1,0,0,0,0,5,12,11,10,6,8,19,20,7,6,54,46,68,17,H


In [5]:
team_away = current_match['home_team_api_id'].values[0]
cm_date = current_match['date'].values[0]
print(team_away)
print(cm_date)
# 10 chelsea's game before the match
def get_n_last_game(n, team_name, curr_date):
    n = 10
    df = df_EPL
    df = df[df['date'] < curr_date]
    df_away = df[df['away_team_api_id'] == team_name]
    df_home = df[df['home_team_api_id'] == team_name]
    df = pd.concat([df_away,df_home])
    return df.sort_values('date',ascending = False)[:n]

get_n_last_game(10,team_away,cm_date)

8455
2016-05-15 00:00:00


,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2894,4623,2015/2016,30,2016-05-11 00:00:00,1989004,8650,8455,1,1,2.00,3.80,3.80,3,0,1,0,11,12,17,9,10,3,10,6,7,3,55,45,31,44,H
2967,4696,2015/2016,37,2016-05-07 00:00:00,1989077,8472,8455,3,2,3.00,3.50,2.45,4,0,3,0,11,9,7,7,4,0,21,18,3,3,31,69,41,41,D
2951,4680,2015/2016,36,2016-05-02 00:00:00,1989061,8455,8586,2,2,2.75,3.60,2.63,3,0,9,0,9,21,5,6,7,8,15,14,7,8,47,53,48,69,D
2941,4670,2015/2016,35,2016-04-23 00:00:00,1989051,8678,8455,1,4,3.10,3.50,2.40,1,0,0,0,11,4,5,4,6,8,22,10,10,7,45,55,55,29,D
2932,4661,2015/2016,34,2016-04-16 00:00:00,1989042,8455,8456,0,3,2.75,3.40,2.70,2,1,3,0,12,15,7,6,6,2,15,5,11,3,51,49,44,57,A
2926,4655,2015/2016,33,2016-04-09 00:00:00,1989036,10003,8455,1,0,3.75,3.40,2.00,5,0,4,0,15,13,8,5,2,8,13,24,1,9,40,60,37,39,D
2911,4640,2015/2016,32,2016-04-02 00:00:00,1989021,10252,8455,0,4,6.50,4.33,1.57,6,0,1,0,12,8,9,4,6,3,11,10,7,2,43,57,51,34,D
2900,4629,2015/2016,31,2016-03-19 00:00:00,1989010,8455,8654,2,2,1.75,3.80,5.25,3,0,5,0,13,14,6,5,15,6,29,7,10,9,57,43,40,49,D
2870,4599,2015/2016,29,2016-03-05 00:00:00,1988990,8455,10194,1,1,1.50,4.50,7.50,1,0,2,0,9,9,11,9,8,6,14,17,8,4,47,53,36,39,D
2864,4593,2015/2016,28,2016-03-01 00:00:00,1988984,9850,8455,1,2,5.25,4.00,1.70,3,0,2,0,17,5,3,5,9,4,27,14,4,4,45,55,24,36,A


In [10]:
# get n last game both team (chelsea and lC)
def get_last_n_matches(current_match, n):
    team_away = current_match['home_team_api_id'].values[0]
    team_home = current_match['away_team_api_id'].values[0]
    cm_date = current_match['date'].values[0]
    df_away = get_n_last_game(n,team_away,cm_date)
    df_home = get_n_last_game(n,team_home,cm_date)
    df = pd.concat([df_home,df_away])
    return df

get_last_n_matches(current_match, 10)

,id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,home_y_card,home_r_card,away_y_card,away_r_card,home_foulcommit,away_foulcommit,home_shoton,away_shoton,home_shotoff,away_shotoff,home_cross,away_cross,home_corner,away_corner,home_possession,away_possession,HTP,ATP,Result
2963,4692,2015/2016,37,2016-05-07 00:00:00,1989073,8197,8668,3,1,1.91,4.00,4.00,0,0,3,0,4,11,15,4,11,2,28,19,12,4,47,53,16,30,D
2953,4682,2015/2016,36,2016-05-01 00:00:00,1989063,10260,8197,1,1,2.10,3.40,3.90,3,0,2,0,16,10,17,4,6,11,38,16,6,5,67,33,40,55,H
2943,4672,2015/2016,35,2016-04-24 00:00:00,1989053,8197,10003,4,0,1.75,3.70,5.50,1,0,2,0,14,10,8,5,6,3,10,22,4,6,40,60,73,40,H
2934,4663,2015/2016,34,2016-04-17 00:00:00,1989044,8197,8654,2,2,1.95,3.60,4.20,3,0,3,0,14,14,0,4,4,11,13,29,0,5,43,57,60,38,D
2925,4654,2015/2016,33,2016-04-10 00:00:00,1989035,8472,8197,0,2,3.40,3.40,2.10,1,0,1,0,12,10,5,8,11,6,11,16,1,7,51,49,51,47,H
2913,4642,2015/2016,32,2016-04-03 00:00:00,1989023,8197,8466,1,0,2.25,3.40,3.50,2,0,2,0,13,13,6,4,3,5,21,36,7,8,47,53,66,47,H
2901,4630,2015/2016,31,2016-03-19 00:00:00,1989011,9826,8197,0,1,3.00,3.40,2.50,1,0,1,0,9,13,8,4,6,4,22,16,10,7,56,44,37,43,A
2893,4622,2015/2016,30,2016-03-14 00:00:00,1989003,8197,10261,1,0,1.57,4.33,6.25,0,0,0,0,4,9,3,4,6,8,19,17,8,4,49,51,60,24,H
2878,4607,2015/2016,29,2016-03-05 00:00:00,1988998,9817,8197,0,1,3.00,3.40,2.55,2,0,2,0,10,11,7,6,5,7,24,13,3,8,50,50,30,24,H
2863,4592,2015/2016,28,2016-03-01 00:00:00,1988983,8197,8659,2,2,1.73,3.80,5.50,0,0,2,0,10,9,11,0,9,5,36,11,11,3,64,36,16,35,A


In [ ]:
df_EPL['Result']

In [ ]:
X = df_EPL.drop(columns = ['Result','season','date','home_team_api_id','away_team_api_id'])
y_cat = pd.Categorical(df_EPL['Result'])
y = y.codes

In [ ]:
print(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(26,))
# h = Conv2D(32, (7, 7), strides=(2, 2))(x)
# h = Activation('relu')(x)
# h = Flatten()(h)
h = Dense(16)(x)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.04)